# Lab 3 — dimensionality reduction

**Group:** W

**Names:**

* Olivier Cloux
* Thibault Urien
* Saskia Reiss

In [ ]:
# Given imports
import json
import matplotlib.pyplot as plt
import numpy as np

# Import panda that is easier to use than dicts
import pandas as pd

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
# Let's see whats inside those files to use them later.
!hdfs dfs -cat /ix/ml-20m/genome-tags.txt | tail -n 1
!hdfs dfs -cat /ix/ml-20m/movies.txt | tail -n 1
!hdfs dfs -cat /ix/ml-20m/genome-scores.txt | tail -n 1
!hdfs dfs -cat /ix/ml-20m/ratings.txt | tail -n 1

In [ ]:
# Given code for the dict of all tag names
data = sc.textFile("/ix/ml-20m/genome-tags.txt").map(json.loads)
#tag2name = dict(data.map(itemgetter("tagId", "tag")).collect())

# Data Frame of all tag names
tag2nameDF = pd.DataFrame(data.map(itemgetter("tagId", "tag")).collect(),columns = ["tagId", "tag"])

# Permits to find longest and shortest tag name 
#tagLen = tag2nameDF.tag.map(len)
#tag2nameDF.loc[tagLen.argmax(), 'tag']
#tagLen = tag2nameDF.tag.map(len)
#tag2nameDF.loc[tagLen.argmin(), 'tag']

### Introduction to the Data
To begin this lab, we choose to answer to the first questions asked in the handout, which are:
* How many different movies are there? How many tags, tag scores, movie ratings?
* How many movies have at least one tag?
* How many different movie genres are there?
* What are the shortest and longest movie titles? Tag names?

Note that we use panda, as we had a few problems with dictionnaries that we could not resolve. The Data Frames given by Panda being quite easy to use too.

In [ ]:
# Data Frame of all movies
data1 = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
moviesDF = pd.DataFrame(data1.map(itemgetter("genres", "movieId", "title")).collect(),columns=["genres", "movieId", "title"])
# Permits us to have all the genres
genresListsDF = pd.DataFrame(data1.map(itemgetter("genres")).collect())
genresDF = pd.DataFrame(genresListsDF.values.flatten()).drop_duplicates().reset_index()

# Permits to find longest and shortest movie name 
#titleLen = moviesDF.title.map(len)
#moviesDF.loc[titleLen.argmax(), 'title']
#titleLen = moviesDF.title.map(len)
#moviesDF.loc[titleLen.argmin(), 'title']

In [ ]:
# Data Frame of all tag scores
data2 = sc.textFile("/ix/ml-20m/genome-scores.txt").map(json.loads)
tagScoresDF = pd.DataFrame(data2.map(itemgetter("relevance", "tagId", "movieId")).collect(), columns=["relevance", "tagId", "movieId"])

# Permits to see how many movies have at least one tag score
filmsWithTag = tagScoresDF.drop_duplicates(subset="movieId")['movieId']

In [ ]:
# This is too big of a file. Need to aggregate more
# Data Frame of all movie ratings

#data3 = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)
#data3.groupBy(lambda x : x[0])
#ratingsDF = pd.DataFrame((data3.groupBy(lambda x : x[0])).collect())


There are 27'278 movies, 20'000'263 ratings, 1'128 tags and 11'709'768 tag scores. There are 19 genres.

10'381 movies have at least one tag, which is a bit less than half of them.

The longest movie title is *Godzilla, Mothra, and King Ghidorah: Giant Monsters All-Out Attack (Gojira, Mosura, Kingu Gidorâ: Daikaijû sôkôgeki) (Godzilla, Mothra and King Ghidorah: Giant Monsters All-Out Attack) (2001)* which is 191 characters long. The shortest title is *M (2001)*.

The longest tag is *oscar (best writing - screenplay written directly for the screen)* and the shotest is *3d*

### Exercise 3.1
MxN matrix

In [ ]:
# Create the M*N tag relevance matrix.
# The tags column is determied by its id.

#  M value. Count number of tag to know dimension of matrix.
tagCount = tagScoresDF.tagId.drop_duplicates().shape[0]
# Debugging line
print("There are",tagCount,"distinct tags")

# List of all movies who have at least one tag. Used to do index table, 
# to be able to map the position of movie in matric to it's Id in the table
tagged_movies_id = tagScoresDF.movieId.drop_duplicates()
# Create index over movieId. Not all movies have a tag, the movie row is determined by its position in a indexing table
tagged_movies_id_index = pd.Index(tagged_movies_id)
# N value.
tagged_movies_count = tagged_movies_id.shape[0]

# Debugging line
print("There are",tagged_movies_count,"movies with at least one tag")

# Empty matrix
tags_mx = np.zeros((tagCount,tagged_movies_count))

# Fill the matrix by iterating over our Panda table y putting the position in the index in the matrix
for tagEntry in tagScoresDF.itertuples():
    tags_mx[tagEntry.tagId-1][tagged_movies_id_index.get_loc(tagEntry.movieId)] = tagEntry.relevance


Plot the variance of each dimension/tag : 
The higher the variance, the more relevant to a film it is, as it is very movie specific. A tag with low variance is 

In [ ]:
#Use variance function to get the variance. Sort it to get cumulative plot.
tag_variance = np.sort(np.var(tags_mx,0))

# Plot the variance by increasing order.
plt.ylabel("variance")
plt.xlabel("tags (sorted by variance)")
plt.plot(tag_variance)
plt.show()

# Get cumulative plot.
values_var, base_var = np.histogram(tag_variance,bins=500,range=[0,max(tag_variance)])
cumulative_var = np.cumsum(values_var)
print(values_var.shape,base_var.shape)
plt.xlabel("Variance")
plt.ylabel("Cumulative % of tags")
plt.plot(base_var[:-1],cumulative_var/cumulative_var[-1]*100)
plt.show()


Plot the eigenvalue of the covariance matrix

In [ ]:
# Use the covariance function.
tag_cov = np.cov(tags_mx)
# Use the eigenvalue function on the covariance.
w,v = np.linalg.eigh(tag_cov)

# Plot the cumulative of the eigenvalues.
# values_eig, base_eig = np.histogram(tag_variance,bins=1000,range=[0,w[-1]])
cumulative_eig = np.cumsum(w)
# print(len(cumulative_eig))


ratio = cumulative_eig/cumulative_eig[-1]
numConcepts = len(ratio[ratio > 1/3])


plt.xlabel("sorted eigenvalue")
plt.ylabel("eigenvalue of covariance")
# plt.subplot(1,1)
plt.plot(w)
plt.yscale('log')
plt.axvline(len(w)-numConcepts)
plt.show()

As we can see above, the end of the curve is really steep on the first graph. This indicates only the first values have significative. To have a better view, we also plot a logplot. We immediately see that taking only few values (here 37), we immediately capture 2/3 of the 'energy'. 

This means that we can capture 2/3 of the dimensions with 37 concepts (instead of 1000). 

### Exercise 3.2 : Concept Space